In [ ]:
import pandas as pd
data = pd.read_csv('https://raw.githubusercontent.com/KurapikaZoldyck/KurapikaZoldyck/main/%23%20-%20Data/user_ratings.csv')
data.head()

In [ ]:
data.info()

In [ ]:
data.shape

In [ ]:
data['title'].nunique()

In [ ]:
import matplotlib.pyplot as plt

data['rating'].hist()
plt.show()

## User Based Recommendation

In [ ]:
count = data['title'].value_counts()
_ = list(count[count > 200].index)
_

In [ ]:
_2 = data[data['title'].isin(_)]
_2

In [ ]:
_3 = round(_2[['title','rating']].groupby('title').mean(),2)
_3.sort_values('rating',ascending=False,inplace=True)
_3

In [ ]:
data['userId'].nunique()

In [ ]:
data[data['userId']==1]

In [ ]:
from itertools import permutations

In [ ]:
?permutations

In [ ]:
def MoviePairs(x):
    pairs = pd.DataFrame(list(permutations(x.values,2)),
                        columns=['movie_a','movie_b'])
    return pairs

In [ ]:
movie_combinations = data.groupby('userId')['title'].apply(MoviePairs)

In [ ]:
movie_combinations

In [ ]:
mc = movie_combinations.groupby(['movie_a','movie_b']).size().to_frame(name='size').reset_index().sort_values(by='size',ascending=False)
mc.head()

In [ ]:
mc[mc['movie_a']=='Forrest Gump (1994)'].head()

In [ ]:
mc[mc['movie_a']=='Forrest Gump (1994)'].tail()

In [ ]:
fg_150 = mc[(mc['movie_a']=='Forrest Gump (1994)')&(mc['size']>=150)]
fg_150.plot.bar(x='movie_b',title='Forrest Gump Movie Recommendation',figsize=(8,8))
plt.show()

## Content Based Recommendation

In [1]:
import pandas as pd
movie = pd.read_csv('https://raw.githubusercontent.com/KurapikaZoldyck/KurapikaZoldyck/main/%23%20-%20Data/movies.csv')
movie.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [2]:
movie.drop('movieId',1,inplace=True)
movie.head()

,title,genres
0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,Jumanji (1995),Adventure|Children|Fantasy
2,Grumpier Old Men (1995),Comedy|Romance
3,Waiting to Exhale (1995),Comedy|Drama|Romance
4,Father of the Bride Part II (1995),Comedy


### Mendapatkan tahun

In [3]:
import re
yr = movie['title'].iloc[0]
yr

'Toy Story (1995)'

In [4]:
year = re.findall("[\d]+",yr)
print(year)
print(type(year))
print(year[0])

['1995']
<class 'list'>
1995


In [5]:
for i in movie['title'].head(2):
    print(re.findall("[\d]+",i))

['1995']
['1995']


In [6]:
movie_year = []

for year in movie['title']:
    yr = re.findall("[\d]+",i)
    movie_year.append(yr[0])

movie['Year'] = movie_year

movie.head(2)

,title,genres,Year
0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1995
1,Jumanji (1995),Adventure|Children|Fantasy,1995


### Normalisasi Title

In [7]:
title = []

for item in movie['title']:
    reg = re.findall("[^\d()]+",item)
    title.append(reg[0].strip())

movie['Title'] = title
movie.drop('title',1,inplace=True)
movie.head(2)

,genres,Year,Title
0,Adventure|Animation|Children|Comedy|Fantasy,1995,Toy Story
1,Adventure|Children|Fantasy,1995,Jumanji


### Normalisasi Genre

### Try Out

In [8]:
co_mo = movie.copy()
co_mo['Genre'] = co_mo['genres'].str.split('|')
co_mo['Genre'].head(2)

0    [Adventure, Animation, Children, Comedy, Fantasy]
1                       [Adventure, Children, Fantasy]
Name: Genre, dtype: object

In [9]:
co_mo.drop('genres',1,inplace=True)
co_mo.head(2)

,Year,Title,Genre
0,1995,Toy Story,"[Adventure, Animation, Children, Comedy, Fantasy]"
1,1995,Jumanji,"[Adventure, Children, Fantasy]"


In [10]:
for index, row in co_mo.head(3).iterrows():
    print('-'*40)
    print(index)
    print('-'*10)
    print(row)

----------------------------------------
0
----------
Year                                                  1995
Title                                            Toy Story
Genre    [Adventure, Animation, Children, Comedy, Fantasy]
Name: 0, dtype: object
----------------------------------------
1
----------
Year                               1995
Title                           Jumanji
Genre    [Adventure, Children, Fantasy]
Name: 1, dtype: object
----------------------------------------
2
----------
Year                  1995
Title     Grumpier Old Men
Genre    [Comedy, Romance]
Name: 2, dtype: object


In [11]:
for index, row in co_mo.head(3).iterrows():
    for genre in row['Genre']:
        print(genre)

Adventure
Animation
Children
Comedy
Fantasy
Adventure
Children
Fantasy
Comedy
Romance


In [12]:
for index, row in co_mo.head(3).iterrows():
    for genre in row['Genre']:
        co_mo.loc[index,genre] = 1
co_mo.head()

,Year,Title,Genre,Adventure,Animation,Children,Comedy,Fantasy,Romance
0,1995,Toy Story,"[Adventure, Animation, Children, Comedy, Fantasy]",1.0,1.0,1.0,1.0,1.0,NaN
1,1995,Jumanji,"[Adventure, Children, Fantasy]",1.0,NaN,1.0,NaN,1.0,NaN
2,1995,Grumpier Old Men,"[Comedy, Romance]",NaN,NaN,NaN,1.0,NaN,1.0
3,1995,Waiting to Exhale,"[Comedy, Drama, Romance]",NaN,NaN,NaN,NaN,NaN,NaN
4,1995,Father of the Bride Part II,[Comedy],NaN,NaN,NaN,NaN,NaN,NaN


### Genre Normalization

In [13]:
movie['Genre'] = movie.genres.str.split('|')
movie.drop('genres',1,inplace=True)
movie.head()

,Year,Title,Genre
0,1995,Toy Story,"[Adventure, Animation, Children, Comedy, Fantasy]"
1,1995,Jumanji,"[Adventure, Children, Fantasy]"
2,1995,Grumpier Old Men,"[Comedy, Romance]"
3,1995,Waiting to Exhale,"[Comedy, Drama, Romance]"
4,1995,Father of the Bride Part II,[Comedy]


In [14]:
for index,row in movie.iterrows():
    for genre in row['Genre']:
        movie.loc[index, genre] = 1
    
movie = movie.fillna(0)
movie.head(2)

,Year,Title,Genre,Adventure,Animation,Children,Comedy,Fantasy,Romance,Drama,...,Horror,Mystery,Sci-Fi,War,Musical,Documentary,IMAX,Western,Film-Noir,(no genres listed)
0,1995,Toy Story,"[Adventure, Animation, Children, Comedy, Fantasy]",1.0,1.0,1.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1995,Jumanji,"[Adventure, Children, Fantasy]",1.0,0.0,1.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [15]:
drop = list(movie[movie['(no genres listed)']==1].index)
movie.drop(drop,inplace=True)
movie.drop('(no genres listed)',1,inplace=True)
movie.head(2)

,Year,Title,Genre,Adventure,Animation,Children,Comedy,Fantasy,Romance,Drama,...,Thriller,Horror,Mystery,Sci-Fi,War,Musical,Documentary,IMAX,Western,Film-Noir
0,1995,Toy Story,"[Adventure, Animation, Children, Comedy, Fantasy]",1.0,1.0,1.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1995,Jumanji,"[Adventure, Children, Fantasy]",1.0,0.0,1.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [16]:
it = list(movie.select_dtypes('float').columns)

for col in it:
    movie[it] = movie[it].astype('int')
movie.head(2)

,Year,Title,Genre,Adventure,Animation,Children,Comedy,Fantasy,Romance,Drama,...,Thriller,Horror,Mystery,Sci-Fi,War,Musical,Documentary,IMAX,Western,Film-Noir
0,1995,Toy Story,"[Adventure, Animation, Children, Comedy, Fantasy]",1,1,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1995,Jumanji,"[Adventure, Children, Fantasy]",1,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


### Jaccard Similarity

In [17]:
import numpy as np
from sklearn.metrics import jaccard_score

In [18]:
mov_genre_df = movie.drop(['Year','Genre'],1)
mov_genre_df.head()

,Title,Adventure,Animation,Children,Comedy,Fantasy,Romance,Drama,Action,Crime,Thriller,Horror,Mystery,Sci-Fi,War,Musical,Documentary,IMAX,Western,Film-Noir
0,Toy Story,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Jumanji,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Grumpier Old Men,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Waiting to Exhale,0,0,0,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0
4,Father of the Bride Part II,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [19]:
mov_genre = movie.drop(['Year','Genre'],1)
mov_genre.drop_duplicates('Title',inplace=True)
mov_genre.set_index('Title',inplace=True)
mov_genre.head()

,Adventure,Animation,Children,Comedy,Fantasy,Romance,Drama,Action,Crime,Thriller,Horror,Mystery,Sci-Fi,War,Musical,Documentary,IMAX,Western,Film-Noir
Title,,,,,,,,,,,,,,,,,,,
Toy Story,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Jumanji,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Grumpier Old Men,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
Waiting to Exhale,0,0,0,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0
Father of the Bride Part II,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [20]:
mov_genre.shape

(9119, 19)

In [21]:
jumanji_values = mov_genre.loc['Jumanji'].values
toy_story_values = mov_genre.loc['Toy Story'].values

In [22]:
jumanji_values

array([1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [23]:
toy_story_values

array([1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [24]:
print("Jumanji : ",sum(list(jumanji_values)))
print("Toy Story : ",sum(list(toy_story_values)))
print(jaccard_score(jumanji_values, toy_story_values))

Jumanji :  3
Toy Story :  5
0.6


In [25]:
from scipy.spatial.distance import pdist,squareform

In [26]:
pdistance = pdist(mov_genre.values, metric='jaccard')
pdistance

array([0.4       , 0.83333333, 0.85714286, ..., 1.        , 1.        ,
       1.        ])

In [27]:
pdistance.shape

(41573521,)

In [31]:
square = 1 - squareform(pdistance)
square

array([[1.        , 0.6       , 0.16666667, ..., 0.        , 0.16666667,
        0.2       ],
       [0.6       , 1.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.16666667, 0.        , 1.        , ..., 0.        , 0.        ,
        0.5       ],
       ...,
       [0.        , 0.        , 0.        , ..., 1.        , 0.        ,
        0.        ],
       [0.16666667, 0.        , 0.        , ..., 0.        , 1.        ,
        0.        ],
       [0.2       , 0.        , 0.5       , ..., 0.        , 0.        ,
        1.        ]])

In [32]:
square.shape

(9119, 9119)

In [33]:
len(mov_genre.index)

9119

In [37]:
jaccard_score = pd.DataFrame(data=np.round(square,2),index=mov_genre.index,columns=mov_genre.index)
jaccard_score.head()

Title,Toy Story,Jumanji,Grumpier Old Men,Waiting to Exhale,Father of the Bride Part II,Heat,Sabrina,Tom and Huck,Sudden Death,GoldenEye,...,Gintama: The Movie,anohana: The Flower We Saw That Day - The Movie,Silver Spoon,Love Live! The School Idol Movie,Jon Stewart Has Left the Building,Black Butler: Book of the Atlantic,No Game No Life: Zero,Flint,Bungo Stray Dogs: Dead Apple,Andrew Dice Clay: Dice Rules
Title,,,,,,,,,,,,,,,,,,,,,
Toy Story,1.00,0.6,0.17,0.14,0.20,0.0,0.17,0.40,0.0,0.14,...,0.29,0.17,0.17,0.2,0.0,0.50,0.60,0.00,0.17,0.20
Jumanji,0.60,1.0,0.00,0.00,0.00,0.0,0.00,0.67,0.0,0.20,...,0.00,0.00,0.00,0.0,0.0,0.17,0.20,0.00,0.00,0.00
Grumpier Old Men,0.17,0.0,1.00,0.67,0.50,0.0,1.00,0.00,0.0,0.00,...,0.20,0.00,0.33,0.0,0.0,0.20,0.25,0.00,0.00,0.50
Waiting to Exhale,0.14,0.0,0.67,1.00,0.33,0.0,0.67,0.00,0.0,0.00,...,0.17,0.25,0.67,0.0,0.0,0.17,0.20,0.33,0.00,0.33
Father of the Bride Part II,0.20,0.0,0.50,0.33,1.00,0.0,0.50,0.00,0.0,0.00,...,0.25,0.00,0.50,0.0,0.0,0.25,0.33,0.00,0.00,1.00


In [40]:
ts = jaccard_score.loc['Toy Story'].sort_values(ascending=False)
jm = jaccard_score.loc['Jumanji'].sort_values(ascending=False)

ts.head()

Title
Toy Story                    1.0
Monsters, Inc.               1.0
Antz                         1.0
Emperor's New Groove, The    1.0
Asterix and the Vikings      1.0
Name: Toy Story, dtype: float64

In [41]:
ts.tail()

Title
Owning Mahowny            0.0
Den, The                  0.0
Shape of Things, The      0.0
Beat Street               0.0
Jesus Christ Superstar    0.0
Name: Toy Story, dtype: float64

In [43]:
jm.head()

Title
Chronicles of Narnia: The Voyage of the Dawn Treader, The    1.0
The Cave of the Golden Rose                                  1.0
Escape to Witch Mountain                                     1.0
Bridge to Terabithia                                         1.0
Santa Claus: The Movie                                       1.0
Name: Jumanji, dtype: float64

In [44]:
jm.tail()

Title
Extremes                  0.0
Edukators, The            0.0
Cop and ½                 0.0
At the Circus             0.0
Jesus Christ Superstar    0.0
Name: Jumanji, dtype: float64

### Text Based Similarity